In [1]:
# Continuing off of #3 web scraping notebook, but using selenium from the beginning

# Getting info scraped from one link

In [2]:
# Import Packages
import time, random
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# Selenium setup
import os
from selenium import webdriver
chromedriver="/Applications/chromedriver"
os.environ["webdriver.chrome.driver"]=chromedriver
driver=webdriver.Chrome(chromedriver)

/var/folders/tj/465rg90j3m59mm_rhyq5qcx40000gn/T/ipykernel_23001/1436372225.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromedriver)


In [3]:
pd.set_option("display.max_columns", None)

In [4]:
# Base link

#url='https://www.baseball-reference.com/boxes/TOR/TOR202209120.shtml'
url='https://www.baseball-reference.com/boxes/ANA/ANA202204070.shtml'


# Will end up looping through every boxscore link

In [5]:
# Scraping single base link
time.sleep(1+2*random.random())
driver.get(url)
time.sleep(1+2*random.random())
html=driver.page_source
soup=bs(html)

In [6]:
# Grab the entire scorebox
scorebox=soup.find('table',{'class':'linescore nohover stats_table no_freeze'}).find_all('tr')

In [7]:
# Separate the three rows of interest
scorebox_row1=scorebox[0].find_all('th')
scorebox_row2=scorebox[1].find_all('td')
scorebox_row3=scorebox[2].find_all('td')

In [8]:
# Create dataframe that will be the scorebox

# set first column name to Team
scorebox_column_names=['Team']

# Grab column names and add to the list
for tag in scorebox_row1[2:]:
    scorebox_column_names.append(tag.text)

In [9]:
# Grab values of row 2
row2_values=[]
for tag in scorebox_row2[1:]:
    row2_values.append(tag.text)

In [10]:
# Grab values of row 3
row3_values=[]
for tag in scorebox_row3[1:]:
    row3_values.append(tag.text)

In [11]:
# Make rows as dictionaries
row1=dict(list(zip(scorebox_column_names,row2_values)))
row2=dict(list(zip(scorebox_column_names,row3_values)))

# Save as list of dictionaries
rows=[row1,row2]

In [12]:
# Create the scorebox dataframe using the row dictionaries list
scorebox_df=pd.DataFrame(rows)

In [13]:
# Defining team1 and team2
team1=scorebox_df['Team'][0]
team2=scorebox_df['Team'][1]

In [14]:
# Now create team batting and pitching dataframes

# This finds the 4 batting and pitching tables (2 per team)
batting_pitching_tables=soup.find_all('table',{'class':'sortable stats_table min_width shade_zero now_sortable'})

In [15]:
# Separate the batting and pitching tables

batting_tables=[batting_pitching_tables[0],batting_pitching_tables[1]]
pitching_tables=[batting_pitching_tables[2],batting_pitching_tables[3]]

In [16]:

# Finding the column names for batting table
batting_columns=[]
for tag in batting_tables[0].find('thead').find_all('th'):
    batting_columns.append(tag.text)
batting_columns[0]='Team'

In [17]:
# Find values for the batting table team1
batting_values_team1=[]
batting_values_team1.append(batting_tables[0].find('tfoot').find('th').text)
for tag in batting_tables[0].find('tfoot').find_all('td'):
    batting_values_team1.append(tag.text)
batting_values_team1[0]=team1

In [18]:
# Find values for the batting table team2
batting_values_team2=[]
batting_values_team2.append(batting_tables[1].find('tfoot').find('th').text)
for tag in batting_tables[1].find('tfoot').find_all('td'):
    batting_values_team2.append(tag.text)
batting_values_team2[0]=team2

In [19]:
# making batting dataframe
batting_row_team1=dict(list(zip(batting_columns,batting_values_team1)))
batting_row_team2=dict(list(zip(batting_columns,batting_values_team2)))
batting_rows=[batting_row_team1,batting_row_team2]
batting_df=pd.DataFrame(batting_rows)

In [20]:
# Now pitching tables

# Finding the column names for pitching table
pitching_columns=[]
for tag in pitching_tables[0].find('thead').find_all('th'):
    pitching_columns.append(tag.text)
pitching_columns[0]='Team'

In [21]:
# Find values for the pitching table team1
pitching_values_team1=[]
pitching_values_team1.append(pitching_tables[0].find('tfoot').find('th').text)
for tag in pitching_tables[0].find('tfoot').find_all('td'):
    pitching_values_team1.append(tag.text)
pitching_values_team1[0]=team1

In [22]:
# Find values for the pitching table team2
pitching_values_team2=[]
pitching_values_team2.append(pitching_tables[1].find('tfoot').find('th').text)
for tag in pitching_tables[1].find('tfoot').find_all('td'):
    pitching_values_team2.append(tag.text)
pitching_values_team2[0]=team2

In [23]:
# making pitching dataframe
pitching_row_team1=dict(list(zip(pitching_columns,pitching_values_team1)))
pitching_row_team2=dict(list(zip(pitching_columns,pitching_values_team2)))
pitching_rows=[pitching_row_team1,pitching_row_team2]
pitching_df=pd.DataFrame(pitching_rows)

In [24]:
# Add homeruns hit to batting_df

team1_hr_batting=pitching_df['HR'][1]
team2_hr_batting=pitching_df['HR'][0]

batting_df.insert(4,'HR_b',[team1_hr_batting,team2_hr_batting])

In [25]:
# get runners in scoring position average
team1_risp_string=soup.find('div',{'id':'teamrispvisitor'}).text
team2_risp_string=soup.find('div',{'id':'teamrisphome'}).text

team1_risp_list=re.findall(r'\d+', team1_risp_string)
team2_risp_list=re.findall(r'\d+', team2_risp_string)

team1_risp_hits=int(team1_risp_list[0])
team2_risp_hits=int(team2_risp_list[0])

batting_df.insert(4,'H_RISP',[team1_risp_hits,team2_risp_hits])


try:
    team1_risp_average=int(team1_risp_list[0])/int(team1_risp_list[1])
except ZeroDivisionError:
    team1_risp_average=0


try:
    team2_risp_average=int(team2_risp_list[0])/int(team2_risp_list[1])
except ZeroDivisionError:
    team2_risp_average=0
    
batting_df.insert(4,'BA_RISP',[team1_risp_average,team2_risp_average])
    
    

In [26]:
# get team left on base numbers
team1_lob_string=soup.find('div',{'id':'teamlobvisitor'}).text
team2_lob_string=soup.find('div',{'id':'teamlobhome'}).text

team1_lob_list=re.findall(r'\d+', team1_lob_string)
team2_lob_list=re.findall(r'\d+', team2_lob_string)

team1_lob=int(team1_lob_list[0])
team2_lob=int(team2_lob_list[0])

batting_df.insert(6,'LOB',[team1_lob,team2_lob])

In [27]:
# Figure out who winner was
team1_score=int(scorebox_df['R'][0])
team2_score=int(scorebox_df['R'][1])

if team1_score>team2_score:
    team1_won=1
    team2_won=0
    
    team1_lost=0
    team2_lost=1

    
    
else:
    team1_won=0
    team2_won=1
    
    team1_lost=1
    team2_lost=0

In [28]:
# Add the winner column to scorebox_df
scorebox_df.insert(len(scorebox_df.columns),'Won',[team1_won,team2_won])
scorebox_df.insert(len(scorebox_df.columns),'Lost',[team1_lost,team2_lost])

In [29]:
# want to combine dataframes into one, but some columns are named the same for batting and pitching so figuring
# this out and adjusting the names

for col in batting_columns:
    if col in pitching_columns:
        print(col)

Team
R
H
BB
SO
Pit
Str
WPA
aLI
cWPA
acLI
RE24


In [30]:
batting_df.rename(columns={'R':'R_b','H':'H_b','BB':'BB_b','SO':'SO_b','Pit':'Pit_b','Str':'Str_b','WPA':'WPA_b','aLI':'aLI_b','cWPA':'cWPA_b','acLI':'acLI_b','RE24':'RE24_b'},inplace=True)

In [31]:
pitching_df.rename(columns={'R':'R_p','H':'H_p','BB':'BB_p','SO':'SO_p','HR':'HR_p','Pit':'Pit_p','Str':'Str_p','WPA':'WPA_p','aLI':'aLI_p','cWPA':'cWPA_p','acLI':'acLI_p','RE24':'RE24_p'},inplace=True)

In [32]:
batting_plus_pitching_df=batting_df.merge(pitching_df,on='Team')

In [33]:
batting_plus_pitching_df

,Team,AB,R_b,H_b,BA_RISP,H_RISP,LOB,HR_b,RBI,BB_b,SO_b,PA,BA,OBP,SLG,OPS,Pit_b,Str_b,WPA_b,aLI_b,WPA+,WPA-,cWPA_b,acLI_b,RE24_b,PO,A,Details,IP,H_p,R_p,ER,BB_p,SO_p,HR_p,ERA,BF,Pit_p,Str_p,Ctct,StS,StL,GB,FB,LD,Unk,GSc,IR,IS,WPA_p,aLI_p,cWPA_p,acLI_p,RE24_p
0,Houston Astros,35,3,8,0.1,1,8,2,3,3,10,38,.229,.289,.514,.804,140,88,-0.117,0.83,0.535,-0.652%,-0.07%,0.85,-1.8,27,13,,9,4,1,1,1,7,0,1.00,31,111,72,38,15,19,13,9,4,0,75,2,0,0.616,1.27,0.37%,1.29,3.8
1,Los Angeles Angels,29,1,4,0.0,0,3,0,1,1,7,31,.138,.194,.207,.400,111,72,-0.615,1.27,0.350,-0.966%,-0.36%,1.24,-3.8,27,10,,9,8,3,3,3,10,2,3.00,38,140,88,45,17,26,10,15,7,0,60,2,0,0.116,0.83,0.07%,0.81,1.8


In [34]:
scorebox_df

,Team,1,2,3,4,5,6,7,8,9,R,H,E,Won,Lost
0,Houston Astros,0,0,1,0,0,0,0,2,0,3,8,1,1,0
1,Los Angeles Angels,0,0,0,0,0,0,0,1,0,1,4,0,0,1


In [35]:
# add errors, won, and lost to batting_plus_pitching_df

# Get columns of interest from scorebox dataframe
scorebox_df[['Team','E','Won','Lost']]



,Team,E,Won,Lost
0,Houston Astros,1,1,0
1,Los Angeles Angels,0,0,1


In [36]:
final_df=batting_plus_pitching_df.merge(scorebox_df[['Team','E','Won','Lost']],on='Team')

In [37]:
# find date of the game
date=soup.find('div',{'class':'scorebox_meta'}).next.next.text

In [38]:
# add date to final_df
final_df['Date']=date

In [39]:
# edit the columns that involve percentages
# remove the percentages from the inputs
# change column name to include % symbol

final_df['WPA-']=final_df['WPA-'].str.replace('%','')
final_df['cWPA_b']=final_df['cWPA_b'].str.replace('%','')
final_df['cWPA_p']=final_df['cWPA_p'].str.replace('%','')

final_df.rename(columns = {'WPA-':'WPA-%', 'cWPA_b':'cWPA_b%','cWPA_p':'cWPA_p%'}, inplace = True)


In [51]:
# now convert the dataframe columns to right datatypes


final_df = final_df.astype({'AB':int,
                            'R_b':int,
                            'H_b':int,
                            'HR_b':int,
                            'RBI':int,
                            'BB_b':int,
                            'SO_b':int,
                            'PA':int,
                            'BA':float,
                            'OBP':float,
                            'SLG':float,
                            'OPS':float,
                            'Pit_b':int,
                            'Str_b':int,
                            'WPA_b':float,
                            'aLI_b':float,
                            'WPA+':float,
                            'WPA-%':float,
                            'cWPA_b%':float,
                            'acLI_b':float,
                            'RE24_b':float,
                            'PO':int,
                            'A':int,
                            'IP':int,
                            'H_p':int,
                            'R_p':int,
                            'ER':int,
                            'BB_p':int,
                            'SO_p':int,
                            'HR_p':int,
                            'ERA':float,
                            'BF':int,
                            'Pit_p':int,
                            'Str_p':int,
                            'Ctct':int,
                            'StS':int,
                            'StL':int,
                            'GB':int,
                            'FB':int,
                            'LD':int,
                            'Unk':int,
                            'GSc':int,
                            'IR':int,
                            'IS':int,
                            'WPA_p':float,
                            'aLI_p':float,
                            'cWPA_p%':float,
                            'acLI_p':float,
                            'RE24_p':float,
                            'E':int,
                            'Won':int,
                            'Lost':int,})

final_df['Date']=pd.to_datetime(final_df['Date'])


In [57]:
# rearrange some column orders
# want date, team, won, lost, then the rest


final_df.insert(0, 'Date', final_df.pop('Date'))
final_df.insert(2, 'Won', final_df.pop('Won'))
final_df.insert(3, 'Lost', final_df.pop('Lost'))


In [58]:
# now need to create this dataframe for every link and add to it
final_df

,Date,Team,Won,Lost,AB,R_b,H_b,BA_RISP,H_RISP,LOB,HR_b,RBI,BB_b,SO_b,PA,BA,OBP,SLG,OPS,Pit_b,Str_b,WPA_b,aLI_b,WPA+,WPA-%,cWPA_b%,acLI_b,RE24_b,PO,A,Details,IP,H_p,R_p,ER,BB_p,SO_p,HR_p,ERA,BF,Pit_p,Str_p,Ctct,StS,StL,GB,FB,LD,Unk,GSc,IR,IS,WPA_p,aLI_p,cWPA_p%,acLI_p,RE24_p,E
0,2022-04-07,Houston Astros,1,0,35,3,8,0.1,1,8,2,3,3,10,38,0.229,0.289,0.514,0.804,140,88,-0.117,0.83,0.535,-0.652,-0.07,0.85,-1.8,27,13,,9,4,1,1,1,7,0,1.0,31,111,72,38,15,19,13,9,4,0,75,2,0,0.616,1.27,0.37,1.29,3.8,1
1,2022-04-07,Los Angeles Angels,0,1,29,1,4,0.0,0,3,0,1,1,7,31,0.138,0.194,0.207,0.400,111,72,-0.615,1.27,0.350,-0.966,-0.36,1.24,-3.8,27,10,,9,8,3,3,3,10,2,3.0,38,140,88,45,17,26,10,15,7,0,60,2,0,0.116,0.83,0.07,0.81,1.8,0


In [59]:
final_df.dtypes

Date       datetime64[ns]
Team               object
Won                 int64
Lost                int64
AB                  int64
R_b                 int64
H_b                 int64
BA_RISP           float64
H_RISP              int64
LOB                 int64
HR_b                int64
RBI                 int64
BB_b                int64
SO_b                int64
PA                  int64
BA                float64
OBP               float64
SLG               float64
OPS               float64
Pit_b               int64
Str_b               int64
WPA_b             float64
aLI_b             float64
WPA+              float64
WPA-%             float64
cWPA_b%           float64
acLI_b            float64
RE24_b            float64
PO                  int64
A                   int64
Details            object
IP                  int64
H_p                 int64
R_p                 int64
ER                  int64
BB_p                int64
SO_p                int64
HR_p                int64
ERA         